In [264]:
# -*- coding: utf-8 -*-
"""
Created on Fri Feb 22 15:45:50 2018
@author: OasisR

"""
import time 
import pandas as pd
import requests
from bs4 import BeautifulSoup
from WindPy import w
from datetime import datetime
import numpy as np
from scipy import interpolate

"""
手动指定计算开始日期

看上次保存在working_data文件夹的“最新结果至xxxx”，将begin_date = "2018-07-11"设为此日期

"""

start_date = "2019-02-20"
# 给出每次运行时当日日期 as enddate
print ('本次运行日期：'+time.strftime("%Y/%m/%d"))
date = time.strftime("%Y-%m-%d")

# 每日有访问限量。。。少访问些比较好

本次运行日期：2019/03/20


In [265]:
"""
如何提升程序性能？

读wind数据的时候用了sleep，原因是读太快存在被封的风险。
最简单尽快读出数据的办法是注释掉sleep，或者改成一个更小的数

"""

'\n如何提升程序性能？\n\n读wind数据的时候用了sleep，原因是读太快存在被封的风险。\n最简单尽快读出数据的办法是注释掉sleep，或者改成一个更小的数\n\n'

In [50]:
"""
连接状况
"""

a = w.start()
if a.ErrorCode==0 : 
    print('\n连接Wind API成功！')


连接Wind API成功！


In [86]:
"""
读取wind中的期权数据

"""

opt = "10001417.SH,10001418.SH,10001419.SH,10001420.SH,10001421.SH,10001422.SH,10001423.SH,10001424.SH,10001425.SH,10001435.SH,10001437.SH,10001439.SH,10001441.SH,10001469.SH,10001501.SH,10001502.SH,10001503.SH,10001504.SH,10001505.SH,10001506.SH,10001507.SH,10001508.SH,10001509.SH,10001519.SH,10001521.SH,10001523.SH,10001579.SH,10001580.SH,10001581.SH,10001582.SH,10001583.SH,10001584.SH,10001585.SH,10001586.SH,10001587.SH,10001597.SH,10001598.SH,10001599.SH,10001600.SH,10001601.SH,10001602.SH,10001603.SH,10001604.SH,10001605.SH,10001619.SH,10001621.SH,10001627.SH,10001629.SH,10001635.SH,10001637.SH,10001643.SH,10001645.SH,10001669.SH,10001671.SH,10001677.SH,10001678.SH,10001679.SH,10001680.SH,10001681.SH,10001682.SH,10001683.SH,10001684.SH,10001685.SH,10001697.SH,10001701.SH,10001705.SH,10001707.SH,10001709.SH,10001713.SH,10001715.SH,10001717.SH,10001727.SH,10001728.SH,10001729.SH,10001730.SH,10001735.SH,10001736.SH,10001737.SH,10001738.SH,10001743.SH,10001744.SH,10001745.SH,10001746.SH,10001751.SH,10001752.SH,10001753.SH,10001754.SH,10001755.SH,10001756.SH,10001757.SH,10001758.SH,10001759.SH,10001769.SH,10001771.SH,10001426.SH,10001427.SH,10001428.SH,10001429.SH,10001430.SH,10001431.SH,10001432.SH,10001433.SH,10001434.SH,10001436.SH,10001438.SH,10001440.SH,10001442.SH,10001470.SH,10001510.SH,10001511.SH,10001512.SH,10001513.SH,10001514.SH,10001515.SH,10001516.SH,10001517.SH,10001518.SH,10001520.SH,10001522.SH,10001524.SH,10001588.SH,10001589.SH,10001590.SH,10001591.SH,10001592.SH,10001593.SH,10001594.SH,10001595.SH,10001596.SH,10001606.SH,10001607.SH,10001608.SH,10001609.SH,10001610.SH,10001611.SH,10001612.SH,10001613.SH,10001614.SH,10001620.SH,10001622.SH,10001628.SH,10001630.SH,10001636.SH,10001638.SH,10001644.SH,10001646.SH,10001670.SH,10001672.SH,10001686.SH,10001687.SH,10001688.SH,10001689.SH,10001690.SH,10001691.SH,10001692.SH,10001693.SH,10001694.SH,10001698.SH,10001702.SH,10001706.SH,10001708.SH,10001710.SH,10001714.SH,10001716.SH,10001718.SH,10001731.SH,10001732.SH,10001733.SH,10001734.SH,10001739.SH,10001740.SH,10001741.SH,10001742.SH,10001747.SH,10001748.SH,10001749.SH,10001750.SH,10001760.SH,10001761.SH,10001762.SH,10001763.SH,10001764.SH,10001765.SH,10001766.SH,10001767.SH,10001768.SH,10001770.SH,10001772.SH"
# 4 weeks before 2019.03.19

opt_list = opt.split(',')
ETF50 = []
for i in opt_list:
    if i[-2:]=='SH':
        ETF50 += [i]

options = pd.DataFrame()
for op in ETF50:
    wsd_data = w.wsd(op, "sec_name,exe_mode,exe_price,exe_enddate,close", "ED-4W", date, "")
    print(wsd_data)
    fm=pd.DataFrame(data=wsd_data.Data, index=wsd_data.Fields, columns=wsd_data.Times)
    fm=fm.T #将矩阵转置
    fm = fm.dropna()
    if len(options)==0:
        options = fm
    else:
        options = pd.concat([options,fm])
    time.sleep(1)

options.to_csv('./raw_data/options'+start_date+'至'+date+'.csv',encoding='GBK')
print('Options Saved！'+start_date+'至'+date)

.ErrorCode=0
.Codes=[10001417.SH]
.Fields=[SEC_NAME,EXE_MODE,EXE_PRICE,EXE_ENDDATE,CLOSE]
.Times=[20190220,20190221,20190222,20190225,20190226,20190227,20190228,20190301,20190304,20190305,...]
.Data=[[50ETF购3月2.352A,50ETF购3月2.352A,50ETF购3月2.352A,50ETF购3月2.352A,50ETF购3月2.352A,50ETF购3月2.352A,50ETF购3月2.352A,50ETF购3月2.352A,50ETF购3月2.352A,50ETF购3月2.352A,...],[认购,认购,认购,认购,认购,认购,认购,认购,认购,认购,...],[2.352,2.352,2.352,2.352,2.352,2.352,2.352,2.352,2.352,2.352,...],[2019-03-27 00:00:00,2019-03-27 00:00:00,2019-03-27 00:00:00,2019-03-27 00:00:00,2019-03-27 00:00:00,2019-03-27 00:00:00,2019-03-27 00:00:00,2019-03-27 00:00:00,2019-03-27 00:00:00,2019-03-27 00:00:00,...],[0.2402,0.2287,0.2769,0.4741,0.3878,0.4027,0.3969,0.4638,0.4722,0.4789,...]]
.ErrorCode=0
.Codes=[10001418.SH]
.Fields=[SEC_NAME,EXE_MODE,EXE_PRICE,EXE_ENDDATE,CLOSE]
.Times=[20190220,20190221,20190222,20190225,20190226,20190227,20190228,20190301,20190304,20190305,...]
.Data=[[50ETF购3月2.401A,50ETF购3月2.401A,50ETF购3月2.401A,50ETF购3月2.401

.ErrorCode=0
.Codes=[10001441.SH]
.Fields=[SEC_NAME,EXE_MODE,EXE_PRICE,EXE_ENDDATE,CLOSE]
.Times=[20190220,20190221,20190222,20190225,20190226,20190227,20190228,20190301,20190304,20190305,...]
.Data=[[50ETF购3月2.156A,50ETF购3月2.156A,50ETF购3月2.156A,50ETF购3月2.156A,50ETF购3月2.156A,50ETF购3月2.156A,50ETF购3月2.156A,50ETF购3月2.156A,50ETF购3月2.156A,50ETF购3月2.156A,...],[认购,认购,认购,认购,认购,认购,认购,认购,认购,认购,...],[2.156,2.156,2.156,2.156,2.156,2.156,2.156,2.156,2.156,2.156,...],[2019-03-27 00:00:00,2019-03-27 00:00:00,2019-03-27 00:00:00,2019-03-27 00:00:00,2019-03-27 00:00:00,2019-03-27 00:00:00,2019-03-27 00:00:00,2019-03-27 00:00:00,2019-03-27 00:00:00,2019-03-27 00:00:00,...],[0.4286,0.4067,0.468,0.666,0.5816,0.5856,0.591,0.6595,0.6697,0.6751,...]]
.ErrorCode=0
.Codes=[10001469.SH]
.Fields=[SEC_NAME,EXE_MODE,EXE_PRICE,EXE_ENDDATE,CLOSE]
.Times=[20190220,20190221,20190222,20190225,20190226,20190227,20190228,20190301,20190304,20190305,...]
.Data=[[50ETF购3月2.794A,50ETF购3月2.794A,50ETF购3月2.794A,50ETF购3月2.794A,5

.ErrorCode=0
.Codes=[10001521.SH]
.Fields=[SEC_NAME,EXE_MODE,EXE_PRICE,EXE_ENDDATE,CLOSE]
.Times=[20190220,20190221,20190222,20190225,20190226,20190227,20190228,20190301,20190304,20190305,...]
.Data=[[50ETF购6月2.205A,50ETF购6月2.205A,50ETF购6月2.205A,50ETF购6月2.205A,50ETF购6月2.205A,50ETF购6月2.205A,50ETF购6月2.205A,50ETF购6月2.205A,50ETF购6月2.205A,50ETF购6月2.205A,...],[认购,认购,认购,认购,认购,认购,认购,认购,认购,认购,...],[2.205,2.205,2.205,2.205,2.205,2.205,2.205,2.205,2.205,2.205,...],[2019-06-26 00:00:00,2019-06-26 00:00:00,2019-06-26 00:00:00,2019-06-26 00:00:00,2019-06-26 00:00:00,2019-06-26 00:00:00,2019-06-26 00:00:00,2019-06-26 00:00:00,2019-06-26 00:00:00,2019-06-26 00:00:00,...],[0.4114,0.4077,0.4497,0.6476,0.573,0.5889,0.5889,0.6517,0.6568,0.6696,...]]
.ErrorCode=0
.Codes=[10001523.SH]
.Fields=[SEC_NAME,EXE_MODE,EXE_PRICE,EXE_ENDDATE,CLOSE]
.Times=[20190220,20190221,20190222,20190225,20190226,20190227,20190228,20190301,20190304,20190305,...]
.Data=[[50ETF购6月2.745A,50ETF购6月2.745A,50ETF购6月2.745A,50ETF购6月2.745A

.ErrorCode=0
.Codes=[10001598.SH]
.Fields=[SEC_NAME,EXE_MODE,EXE_PRICE,EXE_ENDDATE,CLOSE]
.Times=[20190220,20190221,20190222,20190225,20190226,20190227,20190228,20190301,20190304,20190305,...]
.Data=[[50ETF购6月2.30,50ETF购6月2.30,50ETF购6月2.30,50ETF购6月2.30,50ETF购6月2.30,50ETF购6月2.30,50ETF购6月2.30,50ETF购6月2.30,50ETF购6月2.30,50ETF购6月2.30,...],[认购,认购,认购,认购,认购,认购,认购,认购,认购,认购,...],[2.3,2.3,2.3,2.3,2.3,2.3,2.3,2.3,2.3,2.3,...],[2019-06-26 00:00:00,2019-06-26 00:00:00,2019-06-26 00:00:00,2019-06-26 00:00:00,2019-06-26 00:00:00,2019-06-26 00:00:00,2019-06-26 00:00:00,2019-06-26 00:00:00,2019-06-26 00:00:00,2019-06-26 00:00:00,...],[0.3316,0.3248,0.3672,0.5612,0.4874,0.5,0.4917,0.5615,0.5699,0.5798,...]]
.ErrorCode=0
.Codes=[10001599.SH]
.Fields=[SEC_NAME,EXE_MODE,EXE_PRICE,EXE_ENDDATE,CLOSE]
.Times=[20190220,20190221,20190222,20190225,20190226,20190227,20190228,20190301,20190304,20190305,...]
.Data=[[50ETF购6月2.35,50ETF购6月2.35,50ETF购6月2.35,50ETF购6月2.35,50ETF购6月2.35,50ETF购6月2.35,50ETF购6月2.35,50ETF购6月2.

.ErrorCode=0
.Codes=[10001635.SH]
.Fields=[SEC_NAME,EXE_MODE,EXE_PRICE,EXE_ENDDATE,CLOSE]
.Times=[20190220,20190221,20190222,20190225,20190226,20190227,20190228,20190301,20190304,20190305,...]
.Data=[[50ETF购3月2.15,50ETF购3月2.15,50ETF购3月2.15,50ETF购3月2.15,50ETF购3月2.15,50ETF购3月2.15,50ETF购3月2.15,50ETF购3月2.15,50ETF购3月2.15,50ETF购3月2.15,...],[认购,认购,认购,认购,认购,认购,认购,认购,认购,认购,...],[2.15,2.15,2.15,2.15,2.15,2.15,2.15,2.15,2.15,2.15,...],[2019-03-27 00:00:00,2019-03-27 00:00:00,2019-03-27 00:00:00,2019-03-27 00:00:00,2019-03-27 00:00:00,2019-03-27 00:00:00,2019-03-27 00:00:00,2019-03-27 00:00:00,2019-03-27 00:00:00,2019-03-27 00:00:00,...],[0.4328,0.4212,0.4726,0.6707,0.5796,0.5986,0.5991,0.6672,0.6621,0.6787,...]]
.ErrorCode=0
.Codes=[10001637.SH]
.Fields=[SEC_NAME,EXE_MODE,EXE_PRICE,EXE_ENDDATE,CLOSE]
.Times=[20190220,20190221,20190222,20190225,20190226,20190227,20190228,20190301,20190304,20190305,...]
.Data=[[50ETF购6月2.15,50ETF购6月2.15,50ETF购6月2.15,50ETF购6月2.15,50ETF购6月2.15,50ETF购6月2.15,50ETF购6月2.

...

.ErrorCode=0
.Codes=[10001609.SH]
.Fields=[SEC_NAME,EXE_MODE,EXE_PRICE,EXE_ENDDATE,CLOSE]
.Times=[20190220,20190221,20190222,20190225,20190226,20190227,20190228,20190301,20190304,20190305,...]
.Data=[[50ETF沽6月2.40,50ETF沽6月2.40,50ETF沽6月2.40,50ETF沽6月2.40,50ETF沽6月2.40,50ETF沽6月2.40,50ETF沽6月2.40,50ETF沽6月2.40,50ETF沽6月2.40,50ETF沽6月2.40,...],[认沽,认沽,认沽,认沽,认沽,认沽,认沽,认沽,认沽,认沽,...],[2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4,2.4,...],[2019-06-26 00:00:00,2019-06-26 00:00:00,2019-06-26 00:00:00,2019-06-26 00:00:00,2019-06-26 00:00:00,2019-06-26 00:00:00,2019-06-26 00:00:00,2019-06-26 00:00:00,2019-06-26 00:00:00,2019-06-26 00:00:00,...],[0.0534,0.0528,0.0424,0.0404,0.0434,0.0394,0.0329,0.026,0.0263,0.0287,...]]
.ErrorCode=0
.Codes=[10001610.SH]
.Fields=[SEC_NAME,EXE_MODE,EXE_PRICE,EXE_ENDDATE,CLOSE]
.Times=[20190220,20190221,20190222,20190225,20190226,20190227,20190228,20190301,20190304,20190305,...]
.Data=[[50ETF沽6月2.45,50ETF沽6月2.45,50ETF沽6月2.45,50ETF沽6月2.45,50ETF沽6月2.45,50ETF沽6月2.45,50ETF沽6月2.45,50ETF沽6月

.ErrorCode=0
.Codes=[10001644.SH]
.Fields=[SEC_NAME,EXE_MODE,EXE_PRICE,EXE_ENDDATE,CLOSE]
.Times=[20190220,20190221,20190222,20190225,20190226,20190227,20190228,20190301,20190304,20190305,...]
.Data=[[50ETF沽3月2.10,50ETF沽3月2.10,50ETF沽3月2.10,50ETF沽3月2.10,50ETF沽3月2.10,50ETF沽3月2.10,50ETF沽3月2.10,50ETF沽3月2.10,50ETF沽3月2.10,50ETF沽3月2.10,...],[认沽,认沽,认沽,认沽,认沽,认沽,认沽,认沽,认沽,认沽,...],[2.1,2.1,2.1,2.1,2.1,2.1,2.1,2.1,2.1,2.1,...],[2019-03-27 00:00:00,2019-03-27 00:00:00,2019-03-27 00:00:00,2019-03-27 00:00:00,2019-03-27 00:00:00,2019-03-27 00:00:00,2019-03-27 00:00:00,2019-03-27 00:00:00,2019-03-27 00:00:00,2019-03-27 00:00:00,...],[0.0014,0.0012,0.001,0.0019,0.0025,0.0017,0.0013,0.001,0.0012,0.0012,...]]
.ErrorCode=0
.Codes=[10001646.SH]
.Fields=[SEC_NAME,EXE_MODE,EXE_PRICE,EXE_ENDDATE,CLOSE]
.Times=[20190220,20190221,20190222,20190225,20190226,20190227,20190228,20190301,20190304,20190305,...]
.Data=[[50ETF沽6月2.10,50ETF沽6月2.10,50ETF沽6月2.10,50ETF沽6月2.10,50ETF沽6月2.10,50ETF沽6月2.10,50ETF沽6月2.10,50ETF沽6月2

.ErrorCode=0
.Codes=[10001694.SH]
.Fields=[SEC_NAME,EXE_MODE,EXE_PRICE,EXE_ENDDATE,CLOSE]
.Times=[20190220,20190221,20190222,20190225,20190226,20190227,20190228,20190301,20190304,20190305,...]
.Data=[[50ETF沽9月2.60,50ETF沽9月2.60,50ETF沽9月2.60,50ETF沽9月2.60,50ETF沽9月2.60,50ETF沽9月2.60,50ETF沽9月2.60,50ETF沽9月2.60,50ETF沽9月2.60,50ETF沽9月2.60,...],[认沽,认沽,认沽,认沽,认沽,认沽,认沽,认沽,认沽,认沽,...],[2.6,2.6,2.6,2.6,2.6,2.6,2.6,2.6,2.6,2.6,...],[2019-09-25 00:00:00,2019-09-25 00:00:00,2019-09-25 00:00:00,2019-09-25 00:00:00,2019-09-25 00:00:00,2019-09-25 00:00:00,2019-09-25 00:00:00,2019-09-25 00:00:00,2019-09-25 00:00:00,2019-09-25 00:00:00,...],[0.1534,0.1551,0.1326,0.1173,0.123,0.116,0.1065,0.0928,0.0882,0.0925,...]]
.ErrorCode=0
.Codes=[10001698.SH]
.Fields=[SEC_NAME,EXE_MODE,EXE_PRICE,EXE_ENDDATE,CLOSE]
.Times=[20190220,20190221,20190222,20190225,20190226,20190227,20190228,20190301,20190304,20190305,...]
.Data=[[50ETF沽9月2.65,50ETF沽9月2.65,50ETF沽9月2.65,50ETF沽9月2.65,50ETF沽9月2.65,50ETF沽9月2.65,50ETF沽9月2.65,50ETF沽9月2

.ErrorCode=0
.Codes=[10001734.SH]
.Fields=[SEC_NAME,EXE_MODE,EXE_PRICE,EXE_ENDDATE,CLOSE]
.Times=[20190220,20190221,20190222,20190225,20190226,20190227,20190228,20190301,20190304,20190305,...]
.Data=[[50ETF沽3月3.00,50ETF沽3月3.00,50ETF沽3月3.00,50ETF沽3月3.00,50ETF沽3月3.00,50ETF沽3月3.00,50ETF沽3月3.00,50ETF沽3月3.00,50ETF沽3月3.00,50ETF沽3月3.00,...],[认沽,认沽,认沽,认沽,认沽,认沽,认沽,认沽,认沽,认沽,...],[3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...],[2019-03-27 00:00:00,2019-03-27 00:00:00,2019-03-27 00:00:00,2019-03-27 00:00:00,2019-03-27 00:00:00,2019-03-27 00:00:00,2019-03-27 00:00:00,2019-03-27 00:00:00,2019-03-27 00:00:00,2019-03-27 00:00:00,...],[None,None,None,None,0.2879,0.2768,0.273,0.2151,0.2123,0.217,...]]
.ErrorCode=0
.Codes=[10001739.SH]
.Fields=[SEC_NAME,EXE_MODE,EXE_PRICE,EXE_ENDDATE,CLOSE]
.Times=[20190220,20190221,20190222,20190225,20190226,20190227,20190228,20190301,20190304,20190305,...]
.Data=[[50ETF沽6月2.85,50ETF沽6月2.85,50ETF沽6月2.85,50ETF沽6月2.85,50ETF沽6月2.85,50ETF沽6月2.85,50ETF沽6月2.85,50ETF沽6月2.85,50ET

.ErrorCode=0
.Codes=[10001763.SH]
.Fields=[SEC_NAME,EXE_MODE,EXE_PRICE,EXE_ENDDATE,CLOSE]
.Times=[20190220,20190221,20190222,20190225,20190226,20190227,20190228,20190301,20190304,20190305,...]
.Data=[[50ETF沽4月2.70,50ETF沽4月2.70,50ETF沽4月2.70,50ETF沽4月2.70,50ETF沽4月2.70,50ETF沽4月2.70,50ETF沽4月2.70,50ETF沽4月2.70,50ETF沽4月2.70,50ETF沽4月2.70,...],[认沽,认沽,认沽,认沽,认沽,认沽,认沽,认沽,认沽,认沽,...],[2.7,2.7,2.7,2.7,2.7,2.7,2.7,2.7,2.7,2.7,...],[2019-04-24 00:00:00,2019-04-24 00:00:00,2019-04-24 00:00:00,2019-04-24 00:00:00,2019-04-24 00:00:00,2019-04-24 00:00:00,2019-04-24 00:00:00,2019-04-24 00:00:00,2019-04-24 00:00:00,2019-04-24 00:00:00,...],[None,None,None,None,None,None,0.0895,0.0633,0.0672,0.0698,...]]
.ErrorCode=0
.Codes=[10001764.SH]
.Fields=[SEC_NAME,EXE_MODE,EXE_PRICE,EXE_ENDDATE,CLOSE]
.Times=[20190220,20190221,20190222,20190225,20190226,20190227,20190228,20190301,20190304,20190305,...]
.Data=[[50ETF沽4月2.75,50ETF沽4月2.75,50ETF沽4月2.75,50ETF沽4月2.75,50ETF沽4月2.75,50ETF沽4月2.75,50ETF沽4月2.75,50ETF沽4月2.75,50ETF沽

In [130]:
"""
存下来tradeday
"""
raw_trade_day = w.tdays("2019-02-21", "").Data[0] 
trade_day = []

for each in raw_trade_day:
    ymd = each.strftime('%Y/%m/%d')
    trade_day.append(ymd)
    
a={'DateTime':trade_day}

trade_day = pd.DataFrame(data=a)
trade_day.to_csv('./raw_data/tradeday'+start_date+'至'+date+'.csv', encoding ='utf_8_sig',index = False)

print('Trade_Day Saved！'+start_date+'至'+date)

Trade_Day Saved！2019-02-20至2019-03-20


In [297]:
"""
读取wind中的shibor数据

已被wind限制访问量
ErrorCode=-40522017

"""

a = w.start()
if a.ErrorCode==0 : 
    print('连接Wind API成功！')
    
shibor_raw_data = w.wsd("SHIBORON.IR, SHIBOR1W.IR, SHIBOR2W.IR, SHIBOR1M.IR, SHIBOR3M.IR, SHIBOR6M.IR, SHIBOR9M.IR, SHIBOR1Y.IR", "close", "2019-02-20", date, "")

if shibor_raw_data.ErrorCode ==  -40522017:
    print('最近访问过多，WIND限制访问量，需要在EDB里手动下载shibor数据，注意保存成csv格式')
#     print('终止程序，手动下载数据后，先运行module，再运行下一模块')
#     exit()

连接Wind API成功！


In [298]:
"""
如果shibor可以读取
"""
a={'DateTime':shibor_raw_data.Times,
   '1D':shibor_raw_data.Data[0],
   '1W':shibor_raw_data.Data[1],
   '2W':shibor_raw_data.Data[2],
   '1M':shibor_raw_data.Data[3],
   '3M':shibor_raw_data.Data[4],
   '6M':shibor_raw_data.Data[5], # ....之前少了6m
   '9M':shibor_raw_data.Data[6],
   '1Y':shibor_raw_data.Data[7],
  }

shibor_raw_data_pd = pd.DataFrame(data=a)
shibor_raw_data_pd = shibor_raw_data_pd.sort_index(axis = 0,ascending = False)
shibor_raw_data_pd.to_csv('./raw_data/shibor'+start_date+'至'+date+'.csv', encoding ='utf_8_sig',index = False)


In [338]:
"""
读取已经存好的数据
如果已经被禁用访问了，则使用第二行注释中的语句读取手动修改过的shibor

"""


# if data.ErrorCode !=  -40522017:
#    shibor_rate = pd.read_csv('./raw_data/shibor'+start_date+'至'+date+'.csv',index_col=0,encoding='GBK')
options_data = pd.read_csv('./raw_data/options'+start_date+'至'+date+'.csv',index_col=0,encoding = 'GBK')
options_data = options.sort_index()
tradeday = pd.read_csv('./raw_data/tradeday'+start_date+'至'+date+'.csv',encoding='GBK')
shibor_rate = pd.read_csv('./raw_data/shibor'+start_date+'至'+date+'.csv', index_col = 0, encoding = 'GBK')


,1D,1W,2W,1M,3M,6M,9M,1Y
DateTime,,,,,,,,
2019-03-20,2.714,2.711,2.9460,2.8750,2.8330,2.865,2.962,3.088
2019-03-19,2.666,2.729,2.9910,2.8520,2.8230,2.865,2.960,3.086
2019-03-18,2.756,2.709,2.8610,2.8320,2.8140,2.860,2.958,3.082
2019-03-15,2.655,2.728,2.6170,2.8100,2.7990,2.858,2.956,3.077
2019-03-14,2.310,2.598,2.4800,2.7740,2.7770,2.854,2.953,3.072
2019-03-13,2.079,2.493,2.4610,2.7540,2.7680,2.851,2.950,3.066
2019-03-12,2.365,2.601,2.4580,2.7320,2.7610,2.851,2.950,3.066
2019-03-11,2.092,2.395,2.4328,2.7172,2.7565,2.850,2.950,3.060
2019-03-08,2.047,2.388,2.4480,2.7120,2.7550,2.850,2.950,3.057


In [339]:
new_index = []
enddate = []
for each in options_data.index.tolist():
    each = each.strftime("%Y/%m/%d")
    new_index.append(str(each))
    
for each in options_data['EXE_ENDDATE'].tolist():
    each = each.strftime("%Y/%m/%d %H:%M")
    enddate.append(str(each))

options_data = options_data.drop(['EXE_ENDDATE'],axis = 1)
options_data.insert(0,'date',new_index)
options_data.insert(0,'EXE_ENDDATE',enddate)
options_data = options_data.set_index('date')
options_data

,EXE_ENDDATE,SEC_NAME,EXE_MODE,EXE_PRICE,CLOSE
date,,,,,
2019/02/20,2019/03/27 00:00,50ETF购3月2.352A,认购,2.352,0.2402
2019/02/20,2019/03/27 00:00,50ETF购3月2.30,认购,2.3,0.2878
2019/02/20,2019/09/25 00:00,50ETF沽9月2.70,认沽,2.7,0.207
2019/02/20,2019/03/27 00:00,50ETF沽3月2.254A,认沽,2.254,0.0033
2019/02/20,2019/03/27 00:00,50ETF购3月2.35,认购,2.35,0.2419
2019/02/20,2019/03/27 00:00,50ETF购3月2.40,认购,2.4,0.1982
2019/02/20,2019/09/25 00:00,50ETF沽9月2.65,认沽,2.65,0.1801
2019/02/20,2019/03/27 00:00,50ETF购3月2.45,认购,2.45,0.158
2019/02/20,2019/03/27 00:00,50ETF沽3月2.303A,认沽,2.303,0.0056


In [340]:
"""
手动下载的wind数据，读取并clean shibor
"""

# shibor_rate = pd.read_csv('./raw_data/shibor_manual'+start_date+'至'+date+'.csv')
# # 这个地方要去掉encoding gbk

# shibor_rate = shibor_rate.drop(index = 0)
# shibor_rate = shibor_rate.drop(list(shibor_rate.index)[-2:])
# # 去掉读取wind数据时无用的第一行和最后两行

# shibor_rate = shibor_rate.sort_index(axis = 0,ascending = False)
# shibor_rate.set_index('指标名称',inplace=True)
# # 重新设置index


# shibor_rate.to_csv('./working_data/shibor_manual'+start_date+'至'+date+'.csv', index = False)
# shibor_rate = pd.read_csv('./working_data/shibor_manual'+start_date+'至'+date+'.csv',index_col=0)
# # shibor_rate.columns
# shibor_rate

'\n手动下载的wind数据，读取并clean shibor\n'

In [341]:
#==============================================================================
# 开始计算ivix部分
#==============================================================================
def periodsSplineRiskFreeInterestRate(options, date):
    """
    params: options: 计算VIX的当天的options数据用来获取expDate
            date: 计算哪天的VIX
    return：shibor：该date到每个到期日exoDate的risk free rate

    """
    date = datetime.strptime(date,'%Y/%m/%d')
    exp_dates = np.sort(options.EXE_ENDDATE.unique())
    periods = {}
    for epd in exp_dates:
        epd = pd.to_datetime(epd)
        periods[epd] = (epd - date).days*1.0/365.0
    shibor_date = datetime.strptime(shibor_rate.index[0], "%Y-%m-%d") 
    if date >= shibor_date:
        date_str = shibor_rate.index[0]
        shibor_values = shibor_rate.ix[0].values
    else:
        date_str = date.strftime("%Y-%m-%d") 
        shibor_values = shibor_rate.loc[date_str].values
        
    shibor = {}
#     多久的shibor
    period = np.asarray([1.0, 7.0, 14.0, 30.0, 90.0, 180.0, 270.0, 360.0]) / 360.0
    min_period = min(period)
    max_period = max(period)
    for p in periods.keys():
        tmp = periods[p]
        if periods[p] > max_period:
            tmp = max_period * 0.99999
        elif periods[p] < min_period:
            tmp = min_period * 1.00001
        sh = interpolate.spline(period, shibor_values, tmp, order=3)
        # 这个地方的函数一定要scipy==0.18.0才可以使用，以后配环境的时候要注意
        shibor[p] = sh/100.0
    return shibor

In [342]:
# 读取某一个date的options

def getHistDayOptions(vixDate,options_data):
    options_data = options_data.loc[vixDate,:]
    return options_data

In [384]:
def getNearNextOptExpDate(options, vixDate):
    # 找到options中的当月和次月期权到期日；
    # 用这两个期权隐含的未来波动率来插值计算未来30隐含波动率，是为市场恐慌指数VIX；
    # 如果options中的最近到期期权离到期日仅剩1天以内，则抛弃这一期权，改
    # 选择次月期权和次月期权之后第一个到期的期权来计算。
    # 返回的near和next就是用来计算VIX的两个期权的到期日
    """
    params: options: 该date为交易日的所有期权合约的基本信息和价格信息
            vixDate: VIX的计算日期
    return: near: 当月合约到期日（ps：大于1天到期）
            next：次月合约到期日
    """
    vixDate = datetime.strptime(vixDate,'%Y/%m/%d')
    optionsExpDate = list(pd.Series(options.EXE_ENDDATE.values.ravel()).unique())
    optionsExpDate = [datetime.strptime(str(i),'%Y/%m/%d %H:%M') for i in optionsExpDate]
    near = min(optionsExpDate)
    optionsExpDate.remove(near)
    if near.day - vixDate.day < 1:
        near = min(optionsExpDate)
        optionsExpDate.remove(near)
    nt = min(optionsExpDate)
    return near, nt

In [385]:
def getStrikeMinCallMinusPutClosePrice(options):
    # options 中包括计算某日VIX的call和put两种期权，
    # 对每个行权价，计算相应的call和put的价格差的绝对值，
    # 返回这一价格差的绝对值最小的那个行权价，
    # 并返回该行权价对应的call和put期权价格的差
    """
    params:options: 该date为交易日的所有期权合约的基本信息和价格信息
    return: strike: 看涨合约价格-看跌合约价格 的差值的绝对值最小的行权价
            priceDiff: 以及这个差值，这个是用来确定中间行权价的第一步
    """
    call = options[options.EXE_MODE==u"认购"].set_index(u"EXE_PRICE").sort_index()
    put  = options[options.EXE_MODE==u"认沽"].set_index(u"EXE_PRICE").sort_index()
    callMinusPut = call.CLOSE - put.CLOSE
    callMinusPut = callMinusPut.astype('float64')
    # callMinusPut.to_frame()
    strike = abs(callMinusPut).idxmin()
    priceDiff = callMinusPut[strike].min()
    return strike, priceDiff

In [386]:
def calSigmaSquare( options, FF, R, T):
    # 计算某个到期日期权对于VIX的贡献sigma；
    # 输入为期权数据options，FF为forward index price，
    # R为无风险利率， T为期权剩余到期时间
    """
    params: options:该date为交易日的所有期权合约的基本信息和价格信息
            FF: 根据上一步计算得来的strike，然后再计算得到的forward index price， 根据它对所需要的看涨看跌合约进行划分。
                取小于FF的第一个行权价为中间行权价K0， 然后选取大于等于K0的所有看涨合约， 选取小于等于K0的所有看跌合约。
                对行权价为K0的看涨看跌合约，删除看涨合约，不过看跌合约的价格为两者的均值。
            R： 这部分期权合约到期日对应的无风险利率 shibor
            T： 还有多久到期（年化）
    return：Sigma：得到的结果是传入该到期日数据的Sigma
    """
    callAll = options[options.EXE_MODE==u"认购"].set_index(u"EXE_PRICE").sort_index()
    putAll  = options[options.EXE_MODE==u"认沽"].set_index(u"EXE_PRICE").sort_index()
    callAll['deltaK'] = 0.05
    putAll['deltaK']  = 0.05
    
    index = callAll.index
    if len(index) < 3:
        callAll['deltaK'] = index[-1] - index[0]
    else:
        for i in range(1,len(index)-1):
            callAll['deltaK'].ix[index[i]] = (index[i+1]-index[i-1])/2.0
        callAll['deltaK'].ix[index[0]] = index[1]-index[0]
        callAll['deltaK'].ix[index[-1]] = index[-1] - index[-2]
    index = putAll.index
    if len(index) < 3:
        putAll['deltaK'] = index[-1] - index[0]
    else:
        for i in range(1,len(index)-1):
            putAll['deltaK'].ix[index[i]] = (index[i+1]-index[i-1])/2.0
        putAll['deltaK'].ix[index[0]] = index[1]-index[0]
        putAll['deltaK'].ix[index[-1]] = index[-1] - index[-2]
    
    call = callAll[callAll.index > FF]
    put  = putAll[putAll.index < FF]
    FF_idx = FF
    if put.empty:
        FF_idx = call.index[0]
        callComponent = call.CLOSE*call.deltaK/call.index/call.index
        sigma = (sum(callComponent))*np.exp(T*R)*2/T
        sigma = sigma - (FF/FF_idx - 1)**2/T
    elif call.empty:
        FF_idx = put.index[-1]
        putComponent = put.CLOSE*put.deltaK/put.index/put.index
        sigma = (sum(putComponent))*np.exp(T*R)*2/T
        sigma = sigma - (FF/FF_idx - 1)**2/T
    else:
        FF_idx = put.index[-1]
        try:
            if len(putAll.ix[FF_idx].CLOSE.values) > 1:
                put['CLOSE'].iloc[-1] = (putAll.ix[FF_idx].CLOSE.values[1] + callAll.ix[FF_idx].CLOSE.values[0])/2.0
        except:
            put['CLOSE'].iloc[-1] = (putAll.ix[FF_idx].CLOSE + callAll.ix[FF_idx].CLOSE)/2.0

        callComponent = call.CLOSE*call.deltaK/call.index/call.index
        putComponent  = put.CLOSE*put.deltaK/put.index/put.index
        sigma = (sum(callComponent)+sum(putComponent))*np.exp(T*R)*2/T
        sigma = sigma - (FF/FF_idx - 1)**2/T
    return sigma

In [398]:
def changeste(t):
    str_t = t.strftime('%Y/%m/%d ')+'00:00'
    return str_t

In [401]:
def calDayVIX(vixDate):
    # 利用CBOE的计算方法，计算历史某一日的未来30日期权波动率指数VIX
    """
    params：vixDate：计算VIX的日期  '%Y/%m/%d' 字符串格式
    return：VIX结果
    """
    print('!!!!!!!!')
    # 拿取所需期权信息
    options = getHistDayOptions(vixDate,options_data)
    near, nexts = getNearNextOptExpDate(options, vixDate)
    shibor = periodsSplineRiskFreeInterestRate(options, vixDate)
    R_near = shibor[datetime(near.year,near.month,near.day)]
    R_next = shibor[datetime(nexts.year,nexts.month,nexts.day)]
    str_near = changeste(near)
    str_nexts = changeste(nexts)
    optionsNearTerm = options[options.EXE_ENDDATE == str_near]
    optionsNextTerm = options[options.EXE_ENDDATE == str_nexts]
    # time to expiration
    vixDate = datetime.strptime(vixDate,'%Y/%m/%d')
    T_near = (near - vixDate).days/365.0
    T_next = (nexts- vixDate).days/365.0
    # the forward index prices
    nearPriceDiff = getStrikeMinCallMinusPutClosePrice(optionsNearTerm)
    nextPriceDiff = getStrikeMinCallMinusPutClosePrice(optionsNextTerm)
    near_F = nearPriceDiff[0] + np.exp(T_near*R_near)*nearPriceDiff[1]
    next_F = nextPriceDiff[0] + np.exp(T_next*R_next)*nextPriceDiff[1]
    # 计算不同到期日期权对于VIX的贡献
    near_sigma = calSigmaSquare( optionsNearTerm, near_F, R_near, T_near)
    next_sigma = calSigmaSquare(optionsNextTerm, next_F, R_next, T_next)

    # 利用两个不同到期日的期权对VIX的贡献sig1和sig2，
    # 已经相应的期权剩余到期时间T1和T2；
    # 差值得到并返回VIX指数(%)
    w = (T_next - 30.0/365.0)/(T_next - T_near)
    vix = T_near*w*near_sigma + T_next*(1 - w)*next_sigma
    return 100*np.sqrt(abs(vix)*365.0/30.0)


In [402]:
# 这一块相当于main
ivix = []
for day in tradeday['DateTime']:
    tmp = calDayVIX(day)
    ivix.append(tmp)

print('Calculation Finished!!!')

2019/02/21
!!!!!!!!
EXE_PRICE
2.050     0.521
2.100    0.4677
2.150    0.4198
2.156    0.4053
2.200    0.3717
2.205    0.3597
2.250    0.3191
2.254    0.3152
2.300    0.2729
2.303    0.2618
2.350    0.2242
2.352    0.2193
2.400    0.1718
2.401     0.171
2.450     0.122
2.450    0.1223
2.500    0.0713
2.500    0.0723
2.549    0.0239
2.550    0.0235
2.598   -0.0258
2.600    -0.027
2.647   -0.0732
2.650   -0.0768
2.696   -0.1229
2.700   -0.1272
2.745   -0.1713
2.750   -0.1758
2.794   -0.2199
2.800   -0.2254
Name: CLOSE, dtype: object
EXE_PRICE
2.050    0.5210
2.100    0.4677
2.150    0.4198
2.156    0.4053
2.200    0.3717
2.205    0.3597
2.250    0.3191
2.254    0.3152
2.300    0.2729
2.303    0.2618
2.350    0.2242
2.352    0.2193
2.400    0.1718
2.401    0.1710
2.450    0.1220
2.450    0.1223
2.500    0.0713
2.500    0.0723
2.549    0.0239
2.550    0.0235
2.598   -0.0258
2.600   -0.0270
2.647   -0.0732
2.650   -0.0768
2.696   -0.1229
2.700   -0.1272
2.745   -0.1713
2.750   -0.1758
2.794

c:\users\linke\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:40: DeprecationWarning: `spline` is deprecated!
spline is deprecated in scipy 0.19.0, use Bspline class instead.
c:\users\linke\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:25: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
c:\users\linke\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:26: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
c:\users\linke\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:27: DeprecationWarning: 
.ix is depreca

2019/02/22
!!!!!!!!
EXE_PRICE
2.050    0.5758
2.100    0.5216
2.150    0.4715
2.156    0.4668
2.200    0.4217
2.205    0.4172
2.250    0.3735
2.254    0.3689
2.300    0.3223
2.303      0.32
2.350    0.2737
2.352    0.2714
2.400    0.2247
2.401     0.221
2.450     0.174
2.450    0.1751
2.500    0.1239
2.500    0.1221
2.549     0.075
2.550    0.0752
2.598     0.025
2.600    0.0252
2.647   -0.0221
2.650   -0.0262
2.696   -0.0721
2.700   -0.0744
2.745   -0.1192
2.750   -0.1232
2.794   -0.1695
2.800    -0.176
Name: CLOSE, dtype: object
EXE_PRICE
2.050    0.5758
2.100    0.5216
2.150    0.4715
2.156    0.4668
2.200    0.4217
2.205    0.4172
2.250    0.3735
2.254    0.3689
2.300    0.3223
2.303    0.3200
2.350    0.2737
2.352    0.2714
2.400    0.2247
2.401    0.2210
2.450    0.1740
2.450    0.1751
2.500    0.1239
2.500    0.1221
2.549    0.0750
2.550    0.0752
2.598    0.0250
2.600    0.0252
2.647   -0.0221
2.650   -0.0262
2.696   -0.0721
2.700   -0.0744
2.745   -0.1192
2.750   -0.1232
2.794

c:\users\linke\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:26: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
c:\users\linke\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:27: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
c:\users\linke\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:33: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
c:\users\linke\appd

2019/02/25
!!!!!!!!
EXE_PRICE
2.050    0.7706
2.100    0.7214
2.150    0.6688
2.156    0.6639
2.200    0.6173
2.205    0.6174
2.250    0.5736
2.254    0.5563
2.300    0.5305
2.303    0.5191
2.350    0.4702
2.352    0.4673
2.400     0.422
2.401    0.4179
2.450    0.3676
2.450    0.3682
2.500    0.3262
2.500    0.3243
2.549     0.269
2.550    0.2728
2.598    0.2249
2.600    0.2245
2.647     0.174
2.650     0.203
2.696    0.1277
2.700     0.132
2.745    0.0835
2.750     0.083
2.794    0.0358
2.800    0.0282
Name: CLOSE, dtype: object
EXE_PRICE
2.050    0.7706
2.100    0.7214
2.150    0.6688
2.156    0.6639
2.200    0.6173
2.205    0.6174
2.250    0.5736
2.254    0.5563
2.300    0.5305
2.303    0.5191
2.350    0.4702
2.352    0.4673
2.400    0.4220
2.401    0.4179
2.450    0.3676
2.450    0.3682
2.500    0.3262
2.500    0.3243
2.549    0.2690
2.550    0.2728
2.598    0.2249
2.600    0.2245
2.647    0.1740
2.650    0.2030
2.696    0.1277
2.700    0.1320
2.745    0.0835
2.750    0.0830
2.794

...

c:\users\linke\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:26: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
c:\users\linke\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:27: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
c:\users\linke\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:33: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
c:\users\linke\appd

2019/03/20
!!!!!!!!
EXE_PRICE
2.050    0.7446
2.100    0.6937
2.150    0.6507
2.156    0.6407
2.200     0.594
2.205    0.5959
2.250    0.5457
2.254    0.5438
2.300    0.4971
2.303    0.4938
2.350    0.4443
2.352    0.4449
2.400    0.3967
2.401    0.3957
2.450    0.3473
2.450     0.344
2.500    0.2945
2.500    0.2937
2.549    0.2466
2.550    0.2444
2.598    0.1959
2.600     0.196
2.647    0.1474
2.650    0.1422
2.696    0.0974
2.700    0.0955
2.745    0.0499
2.750    0.0443
2.794    0.0004
2.800   -0.0047
2.850   -0.0544
2.900   -0.1056
2.950   -0.1551
3.000   -0.2034
Name: CLOSE, dtype: object
EXE_PRICE
2.050    0.7446
2.100    0.6937
2.150    0.6507
2.156    0.6407
2.200    0.5940
2.205    0.5959
2.250    0.5457
2.254    0.5438
2.300    0.4971
2.303    0.4938
2.350    0.4443
2.352    0.4449
2.400    0.3967
2.401    0.3957
2.450    0.3473
2.450    0.3440
2.500    0.2945
2.500    0.2937
2.549    0.2466
2.550    0.2444
2.598    0.1959
2.600    0.1960
2.647    0.1474
2.650    0.1422
2.696

c:\users\linke\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:26: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
c:\users\linke\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:27: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
c:\users\linke\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:33: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
c:\users\linke\appd

Calculation Finished!!!


c:\users\linke\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:34: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
c:\users\linke\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:35: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
c:\users\linke\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:53: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
c:\users\linke\appd

In [422]:
# 配合读取期权论坛的脚本，可以共同绘图、存储
from pyecharts import Line

date = time.strftime("%Y%m%d")
qVIX = pd.read_csv('./working_data/qvix'+date+".csv",encoding='GBK')
attr = tradeday['DateTime'].tolist()
length = len(attr)

# 只画最近20天
qVIX = qVIX[-length:]

line = Line(u"中国波动率指数")
line.add("期权论坛发布", attr, qVIX['5'].tolist(), mark_point=["max"]) # 用的是收盘价
line.add("手动计算", attr, ivix, mark_line=["max"])
line.render('./figure/'+start_date+'至'+date+'vix.html')

In [423]:
c={'期权论坛发布':qVIX['5'].tolist(),
   '手动计算':ivix}
print(type(c))
all_result = pd.DataFrame(data=c)
all_result.to_csv('./working_data/最终结果'+start_date+'至'+date+'.csv', encoding ='GBK')

<class 'dict'>
